In [1]:
import tensorflow as tf
import nltk
import numpy as np
import os
import time
import pandas as pd
import csv
import string
import re

In [2]:
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))

Num of GPUs available:  13


In [3]:
genre = input("Which genre: ")

Which genre:  Adventure


In [4]:
data = pd.read_csv('games.csv')

In [5]:
data.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [6]:
data2 = pd.DataFrame(data[["Name","Estimated owners","Genres","About the game"]])

In [7]:
data2 = data2[data2['Genres'].str.contains(genre)==True]

In [8]:
data2 = data2[data2['Estimated owners'].str.contains('0 - 20000')==False]

In [9]:
data2 = data2[data2['Estimated owners'].str.contains('0 - 0')==False]

In [10]:
data2

,Name,Estimated owners,Genres,About the game
5,Wartune Reborn,50000 - 100000,"Adventure,Casual,Free to Play,Massively Multip...",Feel tired of auto-fight? Feel tired of boring...
9,Deadlings: Rotten Edition,50000 - 100000,"Action,Adventure,Indie",Death is lonely. He has zero friends on his Fa...
21,Project: R.E.B.O.O.T 2,20000 - 50000,"Action,Adventure,Casual,Indie",Meet the continuation of the hardcore platform...
23,Turtle Lu,20000 - 50000,"Action,Adventure,Casual,Indie",Turtle Lu Turtle Lu is an old-school Action-Ad...
33,Unhappy Ever After,20000 - 50000,"Adventure,Indie,RPG",'In the infinitesimal spaces between the pages...
...,...,...,...,...
71524,Beyond Polaris Guardian,20000 - 50000,"Action,Adventure,RPG,Early Access",Military schools serve as a crucial step for r...
71534,GunSoul Girl 2,50000 - 100000,"Adventure,RPG,Early Access",'GunSoul Girl 2' is a third-person shooting ga...
71542,The Adventures of LinShanHai - Chapter3:The He...,20000 - 50000,"Adventure,Indie,Simulation",第三案:尸色 案件简介: 有人报告一个废弃厂房最近恶臭难闻，有人发现房梁上挂着一具腐烂的尸体...
71585,The Last Worker,200000 - 500000,"Adventure,Indie",The Last Worker is an immersive narrative adve...


In [11]:
data2 = data2['About the game'].str.join('')

In [12]:
data2

5        Feel tired of auto-fight? Feel tired of boring...
9        Death is lonely. He has zero friends on his Fa...
21       Meet the continuation of the hardcore platform...
23       Turtle Lu Turtle Lu is an old-school Action-Ad...
33       'In the infinitesimal spaces between the pages...
                               ...                        
71524    Military schools serve as a crucial step for r...
71534    'GunSoul Girl 2' is a third-person shooting ga...
71542    第三案:尸色 案件简介: 有人报告一个废弃厂房最近恶臭难闻，有人发现房梁上挂着一具腐烂的尸体...
71585    The Last Worker is an immersive narrative adve...
71667    Experience the emotional storytelling and unfo...
Name: About the game, Length: 5710, dtype: object

In [13]:
text = data2.str.cat(sep = ' ')

In [14]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jerra\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [14]:
words = set(nltk.corpus.words.words())
printable = set(string.printable)
text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
text = ''.join(filter(lambda x: x in printable, text))
text = re.sub(r'\s([?.,!"](?:\s|$))', r'\1', text)
text = re.sub(r"\b\s+'\b", r"'", text)

In [15]:
print(f'Length of text: {len(text)} characters')

Length of text: 6548190 characters


In [16]:
print(text[:5000])

Feel tired of auto - fight? Feel tired of boring numerical? Let us try this amazing and exciting game! Right! Reborn is the super popular magic epic game as we know. Now, the magnificent magic epic game, you the classical and great experience. In Cloud City, you are going to the movement, fighting the abyss to protect the world from chaos. Don ' t be nervous! With protection of, Priestess of Light, you will have the power to absorb the chord, incorporate those without into your own, and make contract with and train apotheosis of. You can improve yourself in a series of, wild fighting, occupation and plunder. At the same time, you can run your own ligeance and and ally yourself with other in the world who attack the Chaos. All of you are going to to protect the world from the control and destroy of Drake and bring the brightness back to Cloud City ...... Enter Reborn to enjoy thinking of and the game! 1. simulation, S - semi battle, , occupation & ; make you the best 2. Magical epic and

In [17]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

92 unique characters


In [18]:
print(vocab)

[' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [19]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [20]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [21]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [22]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(6548190,), dtype=int64, numpy=array([36, 67, 67, ..., 74, 67, 14], dtype=int64)>

In [23]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [24]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
e
e
l
 
t
i
r
e
d


In [25]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'e' b'e' b'l' b' ' b't' b'i' b'r' b'e' b'd' b' ' b'o' b'f' b' '
 b'a' b'u' b't' b'o' b' ' b'-' b' ' b'f' b'i' b'g' b'h' b't' b'?' b' '
 b'F' b'e' b'e' b'l' b' ' b't' b'i' b'r' b'e' b'd' b' ' b'o' b'f' b' '
 b'b' b'o' b'r' b'i' b'n' b'g' b' ' b'n' b'u' b'm' b'e' b'r' b'i' b'c'
 b'a' b'l' b'?' b' ' b'L' b'e' b't' b' ' b'u' b's' b' ' b't' b'r' b'y'
 b' ' b't' b'h' b'i' b's' b' ' b'a' b'm' b'a' b'z' b'i' b'n' b'g' b' '
 b'a' b'n' b'd' b' ' b'e' b'x' b'c' b'i' b't' b'i' b'n' b'g' b' ' b'g'
 b'a' b'm' b'e'], shape=(101,), dtype=string)


In [26]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Feel tired of auto - fight? Feel tired of boring numerical? Let us try this amazing and exciting game'
b'! Right! Reborn is the super popular magic epic game as we know. Now, the magnificent magic epic game'
b', you the classical and great experience. In Cloud City, you are going to the movement, fighting the '
b"abyss to protect the world from chaos. Don ' t be nervous! With protection of, Priestess of Light, yo"
b'u will have the power to absorb the chord, incorporate those without into your own, and make contract'


In [27]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [28]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [29]:
dataset = sequences.map(split_input_target)

In [30]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Feel tired of auto - fight? Feel tired of boring numerical? Let us try this amazing and exciting gam'
Target: b'eel tired of auto - fight? Feel tired of boring numerical? Let us try this amazing and exciting game'


In [31]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [32]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [33]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.gru2 = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      state1 = self.gru1.get_initial_state(x)
      state2 = self.gru2.get_initial_state(x)
    else:
       state1, state2 = states
    x, state1 = self.gru1(x, initial_state=state1, training=training)
    x, state2 = self.gru2(x, initial_state=state2, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, [state1, state2]

    if return_state:
      return x, states
    else:
      return x

In [34]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [35]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 93) # (batch_size, sequence_length, vocab_size)


In [36]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23808     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 gru_1 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  95325     
                                                                 
Total params: 10,355,037
Trainable params: 10,355,037
Non-trainable params: 0
_________________________________________________________________


In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [38]:
sampled_indices

array([ 8, 48, 87, 50, 53, 55,  5, 25, 35,  1, 56, 92,  3, 34, 32, 87, 44,
        6, 12, 72, 64, 59, 13, 81, 58, 45, 70, 70, 42, 92, 13,  3, 61,  1,
       33, 23, 63, 74, 13,  8, 47, 87, 24, 27, 19, 16, 21,  4, 12, 76, 72,
       22, 18, 35, 82, 77, 26, 63,  6,  2, 32, 26, 56, 21, 49, 63, 88, 14,
       40, 86, 66, 19, 13, 24, 81, 38, 60, 58, 32, 35, 82, 82, 71, 69, 10,
       91, 17, 52, 39,  5, 62, 50, 43, 52, 73, 79, 47, 67, 43, 37],
      dtype=int64)

In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.532525, shape=(), dtype=float32)


In [41]:
tf.exp(example_batch_mean_loss).numpy()

92.99308

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS = 16

In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/16
1013/1013 [==============================] - 74s 69ms/step - loss: 1.7105
Epoch 2/16
1013/1013 [==============================] - 73s 71ms/step - loss: 1.2701
Epoch 3/16
1013/1013 [==============================] - 70s 69ms/step - loss: 1.2036
Epoch 4/16
1013/1013 [==============================] - 71s 70ms/step - loss: 1.1612
Epoch 5/16
1013/1013 [==============================] - 72s 70ms/step - loss: 1.1269
Epoch 6/16
1013/1013 [==============================] - 71s 69ms/step - loss: 1.0967
Epoch 7/16
1013/1013 [==============================] - 71s 69ms/step - loss: 1.0700
Epoch 8/16
1013/1013 [==============================] - 71s 69ms/step - loss: 1.0455
Epoch 9/16
1013/1013 [==============================] - 71s 69ms/step - loss: 1.0242
Epoch 10/16
1013/1013 [==============================] - 70s 69ms/step - loss: 1.0056
Epoch 11/16
1013/1013 [==============================] - 72s 70ms/step - loss: 0.9899
Epoch 12/16
1013/1013 [==============================] - 71s 69

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
prompt = input("Input prompt: ")
li = list(prompt.split(" "))

Input prompt:  Fantasy


In [49]:
start = time.time()
states = None
next_char = tf.constant(li)
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Fantasy. Explore dark with no office are waiting to be fought. - New Quest Music where up with unknown - launch blick action with over 120 survival and to take the base with certain epic. In addition to like in of several, they never know why - '. Unique twist on the radio, a winder number of attack and weapon, learning and set out into the end Ling early weird. Key Basis of : Fight ( ) and in left editor ( primaryly and coming soon ); in this side - game to earn  Ahood, days, peegron, and more! Solve Hub Wolf! Defeat with the research facility where it mean one of my open - world wandering with a letter looking for three : a story about the sleepy town only by the Spirit Author of the ' and it ' s up to a terrible duaght - here that in other proddle Creative Mode. Content await! Lead the latest crash and find your way out.  See your evil pushen himself in the making with high yet. Unique open world and find the with the to overcome. Complete to Explore 55 and rended to driving back. T